In [1]:
import pandas as pd

players = ['주승우', '하영민', '헤이수스', '후라도']
files = ['_zone_avg.csv', '_zone_rate.csv']

index_list = ['타율', '구사율']
removing_index = [' - 너클볼', ' - 우언', ' - 주자없음', ' - 주자있음', ' - 득점권', ' - 양타', ' - 구종모름']
count_index = [' - 초구', ' - 스트라이크 > 볼', ' - 볼 > 스트라이크', ' - 스트라이크 = 볼']
fast_ball_index = [' - 투심', ' - 포심', ' - 커터']
droping_columns = ['PA_' + str(i+1) for i in range(25)] + ['컨택율', '타율', 'ops', '']

for player in players:
    for i in range(2):
        # 파일 읽기
        df = pd.read_csv('pitcher_data/' + player + files[i])
        df = df.drop(columns=droping_columns, errors='ignore')

        # 제거할 행 필터링 (첫 번째 열의 값이 `removing_index`에 포함된 경우)
        removing_rows = [index_list[i] + index for index in removing_index]
        df = df[~df.iloc[:, 0].isin(removing_rows)]

        # count 행을 합쳐서 하나의 행으로 만들기
        count_rows = [index_list[i] + index for index in count_index]
        if all(row in df.iloc[:, 0].values for row in count_rows):
            count_sum = df[df.iloc[:, 0].isin(count_rows)].iloc[:, 1:].mean()
            count_sum = count_sum.round(3)
            count_sum_row = pd.DataFrame([[index_list[i] + ' - 카운트'] + count_sum.tolist()], columns=df.columns)
            df = pd.concat([df, count_sum_row], ignore_index=True)
            df = df[~df.iloc[:, 0].isin(count_rows)]

        fastball_rows = [index_list[i] + index for index in fast_ball_index]

        # fastball_rows에 있는 항목이 df의 첫 번째 열에 존재하는지 확인
        matching_rows = df[df.iloc[:, 0].isin(fastball_rows)]

        if not matching_rows.empty:
            # 1개 이상의 행이 일치하면 평균을 구해서 합산
            if len(matching_rows) > 1:
                fastball_sum = matching_rows.iloc[:, 1:].mean()  # 두 번째 열부터 평균 계산
                fastball_sum = fastball_sum.round(3)  # 소수점 3자리로 반올림
                fastball_sum_row = pd.DataFrame([[index_list[i] + ' - 패스트볼'] + fastball_sum.tolist()], columns=df.columns)
                df = pd.concat([df, fastball_sum_row], ignore_index=True)
            else:
                # 1개만 일치하는 경우 이름만 바꿈
                matching_rows.iloc[0, 0] = index_list[i] + ' - 패스트볼'
                df = pd.concat([df, matching_rows.iloc[[0]]], ignore_index=True)

            # 처리한 행들 삭제
            df = df[~df.iloc[:, 0].isin(fastball_rows)]

        df.loc[df.iloc[:, 0] == index_list[i] + ' - 2S 이후', df.columns[0]] = index_list[i] + ' - 결정구'
        df.to_csv(player + files[i])
